## Helper Functions and other functions not shown in the main notebook

**Convert .txt files to dataframes (which will be converted to csv afterwards**

In [4]:
def text_to_df (file_path):

    # List to keep dictionaries for each beer
    beers_dic = []

    # A temporary dictionary to store data for each beer
    current_beer = {}

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Split the line using the first colon found
            parts = line.split(':', 1)
            if len(parts) == 2:
                key = parts[0].strip()
                value = parts[1].strip()
                # Add/update the key in the current beer dictionary
                current_beer[key] = value
           # If you encounter an empty line, it signifies the end of a beer record
            if line.strip() == '':
                beers_dic.append(current_beer)
                current_beer = {}

   # Make sure to add the last beer if the file doesn't end with an empty line
    if current_beer:
        beers_dic.append(current_beer)

 # Create a DataFrame from the list of beer dictionaries
    return pd.DataFrame(beers_dic)

**Create maps using geopy**

In [5]:
def get_coordinates(country):
    # Initialize a geolocator using Nominatim with a specific user_agent
    geolocator = Nominatim(user_agent="geoapiExercices")
    try:
        # obtain the location (latitude and longitude) for the given country
        location = geolocator.geocode(country, language='en')
        return (location.latitude,location.longitude)
    except:
        return (None,None)

In [6]:
def plot_map_ratings (user_ratings):
    # Count the number of ratings for each country
    country_counts = user_ratings['country'].value_counts().reset_index()
    # Rename columns
    country_counts.columns = ['country', 'nbr_ratings']
    # Add a new column 'Coordinates'
    country_counts['Coordinates'] = country_counts['country'].apply(get_coordinates)
    # Initialize a Folium map with an initial center at latitude 0 and longitude 0
    m = folium.Map(location=[0,0],zoom_start=2)
    
    # Iterate over each row in the country_counts dataFrame
    for _, row in country_counts.iterrows():
        # Check if coordinates for the country are available
        if row['Coordinates'][0] is not None:
            # Add a Circle marker to the map for each country
            folium.Circle(
                location= row['Coordinates'],
                radius= row['nbr_ratings'],
                color= 'crimson',
                fill= True,
                fill_color= 'crimson',
                popup='{}: {} ratings'.format(row['country'], row['nbr_ratings'])
            ).add_to(m)
            
    return m

**Plot the general trends, the seasonal trends, and the noise**

In [7]:
def plot_STL(ratings_per_month, type):

    # Apply Seasonal-Trend decomposition using LOESS (STL)
    stl = STL(ratings_per_month, seasonal=13, period=12)
    result = stl.fit() # fit the model

    # Extract components from the decomposition
    trend = result.trend
    seasonal = result.seasonal
    residual = result.resid

    # Create 4 subplot figure
    plt.figure(figsize=(10, 6))

    # Subplot 1: Trend
    plt.subplot(411)
    plt.plot(trend, label='Trend', color = type)
    plt.legend(loc='best')

    # Subplot 2: Seasonality
    plt.subplot(412)
    plt.plot(seasonal,label='Seasonality', color = type)
    plt.legend(loc='best')

    # Subplot 3: Residuals
    plt.subplot(413)
    plt.plot(residual, label='Residuals', color = type)
    plt.legend(loc='best')
    plt.tight_layout()